In [1]:
#initialization
import matplotlib.pyplot as plt
import numpy as np
import math

from qiskit import transpile
from qiskit_aer import AerSimulator  # as of 25Mar2025
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT, UnitaryGate, PhaseGate, RZGate
from qiskit.quantum_info import Operator

# import basic plot tools
from qiskit.visualization import plot_histogram

In [2]:
def standard_qpe(unitary: Operator, eigenstate: QuantumCircuit, num_ancilla: int) -> QuantumCircuit:
    """Constructs a standard Quantum Phase Estimation (QPE) circuit using repeated controlled-U applications."""
    num_qubits = unitary.num_qubits
    qc = QuantumCircuit(num_ancilla + num_qubits, num_ancilla)

    # Prepare eigenstate on system qubits
    qc.append(eigenstate, range(num_ancilla, num_ancilla + num_qubits))

    # Apply Hadamard gates to ancilla qubits
    qc.h(range(num_ancilla))

    # Apply controlled-U^(2^k) using repeated controlled applications of U
    for k in range(num_ancilla):
        controlled_U = UnitaryGate(unitary.data).control(1, label=f"U")
        
        # Apply controlled-U 2^k times
        for _ in range(2**k):  
            qc.append(controlled_U, [k] + list(range(num_ancilla, num_ancilla + num_qubits)))

    # Apply inverse QFT on ancilla qubits
    qc.append(QFT(num_ancilla, inverse=True, do_swaps=True), range(num_ancilla))

    # Measure ancilla qubits
    qc.measure(range(num_ancilla), range(num_ancilla))

    return qc

SANITY CHECK:

$$ U = \begin{pmatrix}
                    1 & 0 \\
                    0 & e^{i\theta}
                \end{pmatrix} $$

So, for $|\lambda \rangle = |1\rangle$  we have $U |1\rangle = e^{i\theta} |1\rangle$ 

In happy cases, we have $ \theta = \frac{2 \pi k}{2^n} $ where $k$ is an integer and $n$ is the number of ancilla qubits and the number read in the ancila register willbe exactly $k$.

Experiments:
Let n = 3, so we have 3 ancilla qubits. The possible values of $k$ are $0, 1, 2, 3, 4, 5, 6, 7$.
- $k = 1 \implies \theta = \frac{2 \pi}{8} = \frac{\pi}{4}$
- $k = 2 \implies \theta = \frac{4 \pi}{8} = \frac{\pi}{2}$
- $k = 3 \implies \theta = \frac{6 \pi}{8} = \frac{3\pi}{4}$
- $k = 4 \implies \theta = \frac{8 \pi}{8} = \pi$
- $k = 5 \implies \theta = \frac{10 \pi}{8} = \frac{5\pi}{4}$
- $k = 6 \implies \theta = \frac{12 \pi}{8} = \frac{3\pi}{2}$
- $k = 7 \implies \theta = \frac{14 \pi}{8} = \frac{7\pi}{4}$
...

In [7]:
def test_general_qpe_with_parametrized_phase(phase, expected_bin):
    unitary = Operator(PhaseGate(phase))
    
    eigenstate = QuantumCircuit(1)
    eigenstate.x(0)

    num_ancilla = 3
    shots = 1024
    qc = standard_qpe(unitary, eigenstate, num_ancilla)

    simulator = AerSimulator()
    job = simulator.run(transpile(qc, simulator), shots=shots)
    result = job.result()
    counts = result.get_counts(qc)
    most_probable = max(counts, key=counts.get)
    assert most_probable.startswith(expected_bin), f"Expected prefix {expected_bin}, got {most_probable}"

    plot_histogram(counts).savefig(f"output_general_phase_{round(phase, 3)}.png")

easy_test_cases = [np.pi/4, np.pi/2, 3*np.pi/4]
for phase in easy_test_cases:
    U = Operator(PhaseGate(phase))
    eigenstate = QuantumCircuit(1)
    eigenstate.x(0)
    num_ancilla = 3
    shots = 1024
    qc = standard_qpe(U, eigenstate, num_ancilla)
    qc.draw("mpl").savefig(f"circuit_general_phase_{round(phase, 3)}.png")

    simulator = AerSimulator()
    job = simulator.run(transpile(qc, simulator), shots=shots)
    result = job.result()
    counts = result.get_counts(qc)
    most_probable = max(counts, key=counts.get)
    print(f"Most probable outcome for phase {phase}: {most_probable}")
    print("Estimated phase:", math.pi * 2 * int(most_probable, 2) / (2**3))

Most probable outcome for phase 0.7853981633974483: 001
Estimated phase: 0.7853981633974483
Most probable outcome for phase 1.5707963267948966: 010
Estimated phase: 1.5707963267948966
Most probable outcome for phase 2.356194490192345: 011
Estimated phase: 2.356194490192345
